# BioBERT-based gene extraction

# 1) Install libraries and load dataset

In [15]:
!pip install transformers
print("Success!")
from transformers import pipeline

import pandas as pd
import os
import re
import time
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import torch
print("Import successful!")

Import successful!


In [116]:
# Load BioBERT Genetic NER Model (Using CUDA if available)
biobert_model = pipeline(
    "ner",
    model="alvaroalon2/biobert_genetic_ner",
    tokenizer="alvaroalon2/biobert_genetic_ner",
    device=0 if torch.cuda.is_available() else -1
)
print("BioBERT model loaded successfully!")

Device set to use cuda:0


BioBERT model loaded successfully!


In [117]:
# Set the working directory and file paths
working_directory = "WORKING_DIRECTORY"
input_directory = "INPUT_DIRECTORY"
output_directory = "OUTPUT_DIRECTORY"
articles_file = "articles.csv"
genes_file = "genes.csv"

# Change the working directory
os.chdir(output_directory)
print("Current Working Directory:", os.getcwd())

os.chdir(input_directory)
genes = pd.read_csv(genes_file, header=None)
gene_list = genes[0].tolist()
print("Genes import successful!")

os.chdir(output_directory)
if "full_articles" not in globals():
    full_articles = pd.read_csv(articles_file)
    print(f"Loaded {len(full_articles)} articles from CSV.")
else:
    print("Using preloaded full_articles from memory.")
articles = full_articles.head(100)
print("Article import successful!")
print(f"\nImported {len(articles)} articles with {len(articles.columns)} selected columns.")
print(f"Imported {len(gene_list):,} oncomine genes.")

num_rows = articles.shape[0]
num_columns = articles.shape[1]
os.chdir(working_directory)
print("\nCurrent Working Directory:", os.getcwd())

Genes import successful!
Using preloaded full_articles from memory.
Article import successful!

Imported 100 articles with 9 selected columns.
Imported 161 oncomine genes.

Current Working Directory: /data/JH/marie/TrendyVariants/ICIMTH


# 2) Run BioBERT model

In [118]:
##### BioBERT with slinding window
# Function to fetch gene synonyms from MyGene.info API
def get_gene_synonyms(gene_symbol):
    """Fetches known synonyms, including protein products, for a given gene from MyGene.info."""
    url = f"https://mygene.info/v3/query?q={gene_symbol}&fields=symbol,alias,other_names"
    try:
        response = requests.get(url).json()
        synonyms = set()
        for hit in response.get("hits", []):
            if "symbol" in hit:
                synonyms.add(hit["symbol"].upper())
            if "alias" in hit:
                synonyms.update([alias.upper() for alias in hit["alias"]])
            if "other_names" in hit:
                synonyms.update([name.upper() for name in hit["other_names"]])
        return synonyms
    except:
        return {gene_symbol.upper()} 

# Expand the gene list dynamically with synonyms
expanded_gene_list = {gene.upper(): get_gene_synonyms(gene) for gene in gene_list}
print(f"Expanded gene list contains {len(expanded_gene_list)} genes with synonyms.")

# Function to normalize extracted genes
def normalize_extracted_genes(found_terms):
    """Normalize and map extracted entities to closest known gene or protein names."""
    normalized_genes = set()
    for term in found_terms:
        term_upper = term.upper()
        if term_upper in expanded_gene_list:
            normalized_genes.add(term_upper)
            continue
        cleaned_term = re.sub(r"[\[\]\(\),-]", " ", term_upper)
        cleaned_words = cleaned_term.split()
        for word in cleaned_words:
            if word in expanded_gene_list:
                normalized_genes.add(word)
        if not any(gene in normalized_genes for gene in cleaned_words):
            match = process.extractOne(term_upper, expanded_gene_list.keys(), scorer=fuzz.ratio)
            if match:
                best_match, score = match[:2]
                if score > 85:
                    normalized_genes.add(best_match)
    return normalized_genes

# Function to split text into overlapping chunks for NER
def sliding_window_chunking(text, tokenizer, max_tokens=512, stride=256):
    """Splits text into overlapping chunks to avoid losing context."""
    tokens = tokenizer.encode(text, add_special_tokens=False)
    if len(tokens) <= max_tokens:
        return [tokenizer.decode(tokens, skip_special_tokens=True)]
    chunks = []
    for i in range(0, len(tokens), stride):
        chunk = tokens[i : i + max_tokens]
        if len(chunk) < max_tokens:
            break
        chunks.append(tokenizer.decode(chunk, skip_special_tokens=True))
    return chunks

# Function to process text with BioBERT using sliding window
def process_biobert(text, model):
    """Runs BioBERT NER with sliding window chunking."""
    if pd.isna(text) or len(text.strip()) == 0:
        return set()
    tokenizer = model.tokenizer
    text_chunks = sliding_window_chunking(text, tokenizer)
    found_terms = set()
    for chunk in text_chunks:
        results = model(chunk)
        current_term = []
        for res in results:
            word = res["word"].replace("##", "")
            if res["entity"].startswith("B-"):
                if current_term:
                    full_term = "".join(current_term)
                    found_terms.add(full_term)
                current_term = [word]
            elif res["entity"].startswith("I-"):
                current_term.append(word)
        if current_term:
            full_term = "".join(current_term)
            found_terms.add(full_term)
    return normalize_extracted_genes(found_terms)

print("Success!")


##### Gene extraction #####
start_time = time.strftime("%Y-%m-%d %H:%M:%S")
start_timestamp = time.time()
print(f"Processing {len(article_df)} articles with BioBERT. Started at {start_time}")

biobert_results = []
for index, row in tqdm(article_df.iterrows(), total=len(article_df), desc="Processing Articles"):
    title = row.get("PaperTitle", "")
    abstract = row.get("Abstract", "")
    genes_biobert = process_biobert(title, biobert_model) | process_biobert(abstract, biobert_model)
    biobert_results.append(", ".join(genes_biobert))
    print(f"Article {index+1}: {genes_biobert}")

df_results = article_df.copy()
df_results["BioBERT"] = biobert_results
num_articles = len(df_results)

output_file = f"filtered_articles_biobert_expanded_{num_articles}.csv"
runtime_file = f"filtered_articles_biobert_expanded_{num_articles}_runtime.txt"

end_time = time.strftime("%Y-%m-%d %H:%M:%S")
end_timestamp = time.time()
total_runtime = end_timestamp - start_timestamp
print(f"Processing completed at {end_time}. Total runtime: {total_runtime:.2f} seconds.")

with open(runtime_file, "w") as f:
    f.write(f"Processing of articles: {num_articles}\n")
    f.write(f"Processing started at: {start_time}\n")
    f.write(f"Processing completed at: {end_time}\n")
    f.write(f"Total runtime: {total_runtime:.2f} seconds\n")
print(f"Runtime details saved in: {runtime_file}")

Expanded gene list contains 161 genes with synonyms.
Success!
Processing 100 articles with BioBERT. Started at 2025-03-06 12:08:39


Processing Articles:   0%|                                                                                  | 0/100 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1355 > 512). Running this sequence through the model will result in indexing errors


Article 1: set()


Processing Articles:   4%|██▉                                                                       | 4/100 [00:00<00:10,  9.13it/s]

Article 2: set()
Article 3: set()
Article 4: set()
Article 5: set()


...

Article 91: {'EZH2', 'KRAS'}
Article 92: {'SRC'}
Article 93: set()
Article 94: set()


Processing Articles:  98%|███████████████████████████████████████████████████████████████████████▌ | 98/100 [00:06<00:00, 15.95it/s]

Article 95: set()
Article 96: set()
Article 97: set()
Article 98: set()


Processing Articles: 100%|████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.46it/s]

Article 99: set()
Article 100: set()
Processing completed at 2025-03-06 12:08:46. Total runtime: 6.92 seconds.
Runtime details saved in: filtered_articles_biobert_expanded_100_runtime.txt


## 3) Create binary matrix

In [122]:
# Binary matrix creation
BioBERT_df = df_results.copy()
print("Length of dataset:", len(BioBERT_df))
print("Column length of dataset:", len(BioBERT_df.columns))
print("Columns of dataset:", BioBERT_df.columns)
BioBERT_original = BioBERT_df["BioBERT"].copy()
BioBERT_df["BioBERT"] = BioBERT_df["BioBERT"].fillna("").astype(str)
BioBERT_df["Extracted_Genes"] = BioBERT_df["BioBERT"].apply(lambda x: [gene.strip() for gene in x.split(',') if gene.strip()])
binary_gene_data = {gene: BioBERT_df["Extracted_Genes"].apply(lambda genes: 1 if gene in genes else 0) for gene in gene_list}
binary_gene_df = pd.DataFrame(binary_gene_data)
BioBERT_df = pd.concat([BioBERT_df, binary_gene_df], axis=1)

BioBERT_df["Sum_Entity_Mentions"] = binary_gene_df.sum(axis=1)

BioBERT_df["BioBERT"] = BioBERT_original
BioBERT_df.drop(columns=["Extracted_Genes"], inplace=True)

# Save as CSV
os.chdir(working_directory)
output_filename = f"BioBERT_evaluation_results.csv"
BioBERT_df.to_csv(output_filename, index=False)
print(f"\nFile saved as: {output_filename}")

os.chdir(working_directory)
binary_gene_columns = [col for col in BioBERT_df.columns if col in gene_list]
BioBERT_df[binary_gene_columns] = BioBERT_df[binary_gene_columns].apply(pd.to_numeric, errors='coerce')

total_gene_mentions = BioBERT_df["Sum_Entity_Mentions"].sum()
total_binary_sum = BioBERT_df[binary_gene_columns].sum().sum()

results_dict = {
    "Metric": ["Total_Sum_Entity_Mentions", "Total_Binary_Matrix_Sum"],
    "Value": [total_gene_mentions, total_binary_sum]
}

results_df = pd.DataFrame(results_dict)
results_df.to_csv("BiobERT_evaluation_results.txt", sep="\t", index=False)

print(f"Results saved to 'Sum_Entity_Mentions_Evaluations.txt'")
print(f"Total sum of 'Sum_Entity_Mentions' column: {total_gene_mentions}")
print(f"Cross-check: Total sum of all binary matrix values (1s in the matrix): {total_binary_sum}")

Length of dataset: 100
Column length of dataset: 10
Columns of dataset: Index(['PaperId', 'PaperTitle', 'Citations', 'CoFoS', 'Authors', 'Abstract',
       'Language', 'PubYear', 'PubDate', 'BioBERT'],
      dtype='object')

File saved as: BioBERT_evaluation_results.csv
Results saved to 'Sum_Entity_Mentions_Evaluations.txt'
Total sum of 'Sum_Entity_Mentions' column: 30
Cross-check: Total sum of all binary matrix values (1s in the matrix): 30
